In [40]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import geopandas as gpd
import pandas as pd
from shapely import wkt
import seaborn as sns

In [55]:
areas = gpd.read_file(r"C:\Users\olive\Documents\GitHub\MontrealFireSafetyProject\data\LIMADMIN.shp",
                      encoding='latin1', index='geometry')
#areas = areas.set_index(['geometry'], drop=True)

In [56]:
poly = pd.read_csv(r"C:\Users\olive\Documents\GitHub\MontrealFireSafetyProject\data\2020_forecast.csv",
                   encoding='latin1')

geometry = poly['Poly_Key'].apply(wkt.loads)
crs = {'init': 'epsg:2263'} #http://www.spatialreference.org/ref/epsg/2263/
poly = gpd.GeoDataFrame(poly, crs=crs, geometry=geometry)
#poly = poly.set_index(['geometry'], drop=True)
poly = poly.drop(['Poly_Key'], axis=1)

In [59]:
joins = gpd.sjoin(areas, poly, how="right", op="contains")
#sns.heatmap(joins.isnull(), cbar=False)

C:\Users\olive\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\geopandas\tools\sjoin.py:61: UserWarning: CRS of frames being joined does not match!({'init': 'epsg:4326'} != {'init': 'epsg:2263'})
  "(%s != %s)" % (left_df.crs, right_df.crs)
C:\Users\olive\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [60]:
joins.head()

,index_left,MUNID,CODEID,CODEMAMROT,NOM,TYPE,ABREV,NUM,AIRE,PERIM,...,Construction_Year_median_thisyear,Building_area_min_thisyear,Building_area_max_thisyear,Building_area_avg_thisyear,Borough_thisyear,Building_fire_nextyear,CFSAUID_thisyear,Poly_Key_nextyear,2020_incidents_forecast,geometry
index_right,,,,,,,,,,,,,,,,,,,,,
14874,0.0,66023.0,11,REM05,Outremont,Arrondissement,OM,5.0,3813355.723,10836.67,...,1927.5,25.0,2423.0,571.423077,Outremont,NaN,NaN,NaN,0.506993,"POLYGON ((-73.59829 45.51650, -73.59761 45.516..."
14882,0.0,66023.0,11,REM05,Outremont,Arrondissement,OM,5.0,3813355.723,10836.67,...,1928.5,111.0,2376.0,499.700000,Outremont,NaN,NaN,NaN,0.518130,"POLYGON ((-73.59974 45.51555, -73.59982 45.515..."
14932,0.0,66023.0,11,REM05,Outremont,Arrondissement,OM,5.0,3813355.723,10836.67,...,1911.0,185.0,26782.0,1709.000000,Outremont,NaN,NaN,NaN,0.412062,"POLYGON ((-73.60689 45.51592, -73.60745 45.515..."
14894,0.0,66023.0,11,REM05,Outremont,Arrondissement,OM,5.0,3813355.723,10836.67,...,1911.0,207.0,11372.0,809.540541,Outremont,NaN,NaN,NaN,0.851268,"POLYGON ((-73.60155 45.51524, -73.60194 45.514..."
14889,0.0,66023.0,11,REM05,Outremont,Arrondissement,OM,5.0,3813355.723,10836.67,...,1911.0,227.0,11372.0,1116.777778,Outremont,NaN,NaN,NaN,0.403524,"POLYGON ((-73.60108 45.51527, -73.60155 45.515..."


In [62]:
grp = joins.groupby(['NOM']).agg({'2020_incidents_forecast': ['mean']})

In [64]:
grp.columns = grp.columns.get_level_values(0)
grp = grp.rename({'2020_incidents_forecast': 'Forecasted Incident Risk Score'}, axis=1)
grp.sort_values('Forecasted Incident Risk Score', ascending=False)

,Forecasted Incident Risk Score
NOM,
Le Plateau-Mont-Royal,0.777029
Villeray-Saint-Michel-Parc-Extension,0.769236
Côte-des-Neiges-Notre-Dame-de-Grâce,0.741717
Rosemont-La Petite-Patrie,0.732069
Montréal-Nord,0.709441
Le Sud-Ouest,0.692389
Outremont,0.690714
Saint-Léonard,0.680919
Lachine,0.667146


In [65]:
out = pd.merge(grp, areas.set_index(['NOM'], drop=True), left_index=True, right_index=True)

In [66]:
crs = {'init': 'epsg:4326'}
gdf = gpd.GeoDataFrame(out.drop(['geometry'], axis=1), crs=crs, geometry=out['geometry'])

In [68]:
gdf.reset_index().to_file(r"C:\Users\olive\Documents\GitHub\MontrealFireSafetyProject\data/forecasted_fire_risk.shp")